In [11]:
import numpy as np
import pandas as pd
import math

In [12]:
data =  pd.read_csv('https://raw.githubusercontent.com/Mondol007/Datasets/main/Buy%20Computer.csv')
data

,Age,Education,Income,Marital Status,Buy Computer
0,36-55,Master's,High,Single,Yes
1,18-35,High School,Low,Single,No
2,36-55,Master's,Low,Single,Yes
3,18-35,Bachelor's,High,Single,No
4,<18,High School,Low,Single,Yes
5,18-35,Bachelor's,High,Married,No
6,36-55,Bachelor's,Low,Married,No
7,>55,Bachelor's,High,Single,Yes
8,36-55,Master's,Low,Married,No
9,>55,Master's,Low,Married,Yes


In [13]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor '''

        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain

        # for leaf node
        self.value = value

In [14]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''

        # initialize the root of the tree
        self.root = None

        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth

    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree '''

        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)

        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"],
                            left_subtree, right_subtree, best_split["info_gain"])
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)

    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''

        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")

        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain

        # return best split
        return best_split

In [15]:
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''

        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right

    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''

        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain

    def entropy(self, y):
        ''' function to compute entropy '''

        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
        def gini_index(self, y):
          ''' function to compute gini index '''

          class_labels = np.unique(y)
          gini = 0
          for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
          return 1 - gini

    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''

        Y = list(Y)
        return max(Y, key=Y.count)

    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''

        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)

    def fit(self, X, Y):
        ''' function to train the tree '''

        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)

    def predict(self, X):
        ''' function to predict new dataset '''

        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions

    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''

        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [16]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [17]:
!pip install anytree

In [18]:
from anytree import Node, RenderTree
import numpy as np
import pandas as pd
import math


def primaryData(dataset):
    index = list(dataset.columns)
    row_len = len(dataset)
    col_len = len(index)

    return index, row_len, col_len


def find_entropy(dataset, index, row_len, col_len, col):
    DATA = {}
    class_item = set()
    attrr_item = set()

    for mKey, lKey in zip(dataset[index[col]].values, dataset[index[col_len - 1]].values):
        class_item.add(lKey)
        attrr_item.add(mKey)

        if mKey in DATA:
            DATA[mKey]['n'] += 1
            if lKey in DATA[mKey]:
                DATA[mKey][lKey] += 1
            else:
                DATA[mKey][lKey] = 1
        else:
            DATA[mKey] = {'n': 1, lKey: 1}
    totalDict = {}
    totalValue = 0
    for d in attrr_item:
        value = 0
        for s in class_item:
            if s in DATA[d]:
                v = DATA[d][s] / DATA[d]['n']
                value += (v * math.log(v, 2))
            else:
                value = 0
        totalDict[d] = -(value * (DATA[d]['n'] / row_len))
        totalValue += -(value * (DATA[d]['n'] / row_len))

    return totalDict, totalValue


def allEntropy(dataset, index, row_len, col_len):
    _class = {}
    for idd in dataset[index[col_len - 1]].values:
        if idd in _class:
            _class[idd] += 1
        else:
            _class[idd] = 1
    _clsEntropy = 0
    for c in _class:
        val = _class[c] / row_len
        _clsEntropy += -(val * math.log(val, 2))
    _attrEntropy = {}
    _entropy = []
    for i in range(0, col_len - 1):
        _attrEntropy[index[i]], val = find_entropy(dataset, index, row_len, col_len, i)
        _entropy.append(val)
    _infoGain = []
    for i in range(0, col_len - 1):
        _infoGain.append(_clsEntropy - _entropy[i])

    return _attrEntropy, _entropy, _infoGain


def treeGenerate(dataset, _attrEntropy, _infoGain, index, p, d, rl, cl):
    migi = _infoGain.index(max(_infoGain))
    location = dataset.index.tolist()
    if (len(d) == 0):
        d[index[migi]] = Node(index[migi])
    else:
        d[index[migi]] = Node(index[migi], parent=d[p])

    for v in _attrEntropy[index[migi]]:
        d[v] = Node(v, parent=d[index[migi]])
        if _attrEntropy[index[migi]][v] == 0:
            ind = list(dataset[index[migi]]).index(v)
            res = dataset[index[cl - 1]][location[ind]]
            d[res] = Node(res, parent=d[v])
        else:
            newDataset = dataset[dataset[index[migi]] == v]
            newDataset = newDataset.drop(columns=index[migi])
            ind, roln, colln = primaryData(newDataset)
            attEnt, entr, infogain = allEntropy(newDataset, ind, roln, colln)
            d = treeGenerate(newDataset, attEnt, infogain, ind, v, d, roln, colln)
    return d


if __name__ == '__main__':
    dataset = pd.read_csv("https://raw.githubusercontent.com/Mondol007/Datasets/main/Buy%20Computer.csv")
    index, rowLength, colLength = primaryData(dataset)

    allEntropy(dataset, index, rowLength, colLength)
    AttEntropy, Entropy, InformatioGain = allEntropy(dataset, index, rowLength, colLength)
    migi = InformatioGain.index(max(InformatioGain))

    d = {}
    treeGenerate(dataset, AttEntropy, InformatioGain, index, 'null', d, rowLength, colLength)
    for pre, _, node in RenderTree(d[index[migi]]):
        print("%s%s" % (pre, node.name))

Age
├── >55
│   └── Yes
├── <18
│   └── Income
│       ├── Low
│       │   └── Yes
│       └── High
│           └── No
├── 36-55
│   └── Marital Status
│       ├── Single
│       │   └── Yes
│       └── Married
│           └── No
└── 18-35
    └── No
